In [16]:
#from Flex_2 import TrainModel,TrainModel_2, LoadData, GetRegrouppedViews
#from SETTINGS import *
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from math import floor
#import Test
import time
import threading
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import FuncFormatter
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from sklearn.cluster import KMeans
from multiprocessing.dummy import Pool as ThreadPool, freeze_support
from itertools import repeat
from collections import namedtuple
from DataProducer import DataProducer
from SETTINGS import *
from sklearn.preprocessing import MinMaxScaler
from queue import *

STABLE_DELTA = 0.69  # 2 views
DAYS_TO_REMOVE = 30

TRAIN_DELTAS_TO_EXTRACT = 2
TEST_DELTAS_TO_EXTRACT = 2

BATCH_SIZE = 1
TRAIN_STEPS = 40000 #50000
TEST_STEPS = 25000

LEARNING_RATE = 2e-3 #8e-3

SEQUENCE_LENGTH = 26

In [19]:


TO_AVG_COUNT = 10000

# SEQUENCE_LENGTH = 8
# PREDICTION_DELTA = 4
#VIDEO_LEN_LIMIT = 50

#MIN_VIEWS = 100  # 1000 ?

#BATCH_SIZE = 10


#fig, ax = plt.subplots()
#plt.figure(figsize=(12, 5))
#ax.xaxis.set_major_formatter(FormatStrFormatter('%.5f'))


def LoadData(name):
    df = pd.read_csv('data/' + name + '.csv',
                     index_col=['date'],
                     dtype={"video_id": object, "views": np.float64},
                     parse_dates=['date'])
    return df


def DfToList(df):
    listOfVideoViews = list(df.groupby('video_id')['views'].apply(pd.DataFrame.as_matrix))
    return listOfVideoViews


def TrainModel_2(trainVideos, testVideos, seqLen, minPredDelta, maxPredDelta,  minLen, maxLen):
    trainVideos = shuffle(trainVideos)
    testVideos = shuffle(testVideos)

    trainDataProducer = DataProducer(trainVideos, seqLen = seqLen, deltasToExtractCount = TRAIN_DELTAS_TO_EXTRACT, minPredDelta = minPredDelta, maxPredDelta = maxPredDelta,  minLen = minLen, maxLen = maxLen)
    testDataProducer = DataProducer(testVideos, seqLen = seqLen, deltasToExtractCount = TEST_DELTAS_TO_EXTRACT, minPredDelta = minPredDelta, maxPredDelta = maxPredDelta,  minLen = minLen, maxLen = maxLen)

    X = tf.placeholder("float", shape=(None, SEQUENCE_LENGTH), name="Inputs")  # , COLUMNS_COUNT
    Y = tf.placeholder("float", shape=(None, 1), name="Outputs")

    W = tf.Variable(tf.ones([SEQUENCE_LENGTH, 1]), name="weight")  # COLUMNS_COUNT

    pred = tf.reduce_sum(tf.matmul(X, W), axis=1)
    loss = tf.reduce_mean(tf.square(pred / Y - 1))
    grads = tf.gradients(loss, [W])[0]

    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    train = optimizer.minimize(loss)

    init = tf.global_variables_initializer()

    sess = tf.Session()
    sess.run(init)
    losses = []
    koefs = []

    for step in range(TRAIN_STEPS):
        data, lables, lableDeltas = zip(*trainDataProducer.GetNextBatch())
        inputs = np.reshape(data, (BATCH_SIZE, SEQUENCE_LENGTH))
        output = np.reshape(lables, (BATCH_SIZE, 1))
        koef = lables[0] / data
        koefs.append(koef.mean())

        _, curLoss = sess.run([train, loss], feed_dict={X: inputs, Y: output})
        losses.append(curLoss)


        if (step % 9000 == 0):
            #Wval = sess.run(W, feed_dict={X: inputs, Y: output})
            #mnkoef = np.array(koefs).mean()
            #print(mnkoef, Wval)
            #cp = sess.run(pred, feed_dict={X: inputs})
            #pr2 = inputs * mnkoef
            #print(cp, pr2)
            print("AVG loss: ", np.array(losses).mean())
            losses.clear()

    mapeArsTr = []

    for step in range(TEST_STEPS):
        data, lables, lableDeltas = zip(*testDataProducer.GetNextBatch())
        inputs = np.reshape(data, (BATCH_SIZE, SEQUENCE_LENGTH))
        lableDeltas = np.reshape(lableDeltas, (BATCH_SIZE, 1))
        predictedLables = sess.run(pred, feed_dict={X: inputs})
        mapeAr = abs(1 - np.exp(predictedLables) / np.exp(lables))
        mapeArsTr.append(mapeAr.mean())


    mapeArsTr = np.array(mapeArsTr)

    return mapeArsTr.mean(), mapeArsTr.var()

def TrainModel(lr, trainVideos, testVideos, seqLen, minPredDelta, maxPredDelta,  minLen, maxLen):
    trainVideos = shuffle(trainVideos)
    testVideos = shuffle(testVideos)

    trainDataProducer = DataProducer(trainVideos, seqLen = seqLen, deltasToExtractCount = TRAIN_DELTAS_TO_EXTRACT, minPredDelta = minPredDelta, maxPredDelta = maxPredDelta,  minLen = minLen, maxLen = maxLen)
    testDataProducer = DataProducer(testVideos, seqLen = seqLen, deltasToExtractCount = TEST_DELTAS_TO_EXTRACT, minPredDelta = minPredDelta, maxPredDelta = maxPredDelta,  minLen = minLen, maxLen = maxLen)

    lableShifts = tf.placeholder("float", shape=(BATCH_SIZE, 1), name="lablesShift")
    X = tf.placeholder("float", shape=(BATCH_SIZE, SEQUENCE_LENGTH), name="viewsHistoryInput")
    Y = tf.placeholder("float", shape=(BATCH_SIZE, 1), name="Outputs")

    W = tf.Variable(tf.ones([SEQUENCE_LENGTH]), name="weight")
    lsW = tf.Variable(tf.ones([SEQUENCE_LENGTH]), name="lableShiftsWeights")
    B = tf.Variable(tf.ones([1]), name="bias")
    predDeltaKoefs = (B  + lableShifts) * lsW
    combinedWeights = predDeltaKoefs * W
    res = X * combinedWeights
    pred = tf.reduce_sum(res, axis=1)
    loss = tf.reduce_mean(tf.square(pred / Y - 1))
    #grads = tf.gradients(loss, [W])[0]

    optimizer = tf.train.AdamOptimizer(lr)
    train = optimizer.minimize(loss)

    init = tf.global_variables_initializer()

    sess = tf.Session()
    sess.run(init)

    q = []

    for step in range(TRAIN_STEPS):
        data, lables, lableDeltas = zip(*trainDataProducer.GetNextBatch())
        inputs = np.reshape(data, (BATCH_SIZE, SEQUENCE_LENGTH))
        inputLableDeltas = np.reshape(lableDeltas, (BATCH_SIZE, 1))
        inputLableDeltas = ((inputLableDeltas - minPredDelta) / (maxPredDelta -  minPredDelta))

        #inputLableDeltas = inputLableDeltas / (maxPredDelta - 1)

        output = np.reshape(lables, (BATCH_SIZE, 1))

        cp = sess.run(pred, feed_dict={X: inputs, lableShifts: inputLableDeltas})
        lsw_v = sess.run(lsW, feed_dict={X: inputs, Y: output})
        b_v = sess.run(B, feed_dict={X: inputs, Y: output})

        _, curLoss = sess.run([train, loss], feed_dict={X: inputs, Y: output, lableShifts: inputLableDeltas})
        q.append(curLoss)
        # for i in range(10):
        #     _, curLoss_i = sess.run([train, loss], feed_dict={X: inputs, Y: output, lableShifts: lableDeltas})

        #avgLoss = ApproxRollingAverage(avgLoss, curLoss, TO_AVG_COUNT)
        if (step % 10000 == 0):
            print('Step: {} from: {}'.format(step, TRAIN_STEPS))
            print("AVG loss: ", np.array(q).mean())
            q.clear()
            #print("AVG loss: ", np.array(list(q.queue)).mean())
            #print("W = ", sess.run(W))

    mapeArs = []

    for step in range(TEST_STEPS):
        data, lables, lableDeltas = zip(*testDataProducer.GetNextBatch())
        inputs = np.reshape(data, (BATCH_SIZE, SEQUENCE_LENGTH))
        inputLableDeltas = np.reshape(lableDeltas, (BATCH_SIZE, 1))
        inputLableDeltas = ((inputLableDeltas - minPredDelta) / (maxPredDelta -  minPredDelta))

        predictedLables = sess.run(pred, feed_dict={X: inputs, lableShifts: inputLableDeltas})
        mapeAr = abs(1 - np.exp(predictedLables.ravel()) / np.exp(lables))
        mapeArs.append(mapeAr.mean())

    mapeArs = np.array(mapeArs)

    return mapeArs.mean(), mapeArs.var()

def GetRegrouppedViews(trainDf, testDf):
    return DfToList(trainDf), DfToList(testDf)


def AddTrend(x, y):
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    plt.plot(x, p(x), "r--")


def Plot(dataL, step=1):
    rseMean, rseVar, mapeMean, mapeVar = zip(*dataL)
    # calc the trendline
    x = np.arange(start=0, stop=len(mapeMean), step=step)
    y = mapeMean
    plt.title('MAPE mean')
    plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    AddTrend(x, y)
    plt.plot(x, y)
    plt.show()

    x = np.arange(len(mapeVar))
    y = mapeVar
    plt.title('MAPE var')
    AddTrend(x, y)
    plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
    plt.plot(x, y)
    plt.show()

    minInd = np.argmin(mapeMean)

    print("Min at index: {}; val: {}%".format(minInd, x[minInd] * 100.0))



In [2]:
trainDf = LoadData('train')
testDf = LoadData('validation')
trainViewsD, testViewsD = GetRegrouppedViews(trainDf, testDf)

In [3]:
def Run(lr):
    start = time.time()

    erMEan, erStd = TrainModel(lr, trainViewsD, testViewsD, seqLen=SEQUENCE_LENGTH, minPredDelta=1, maxPredDelta=120, 
                               minLen=180, maxLen=None)

    print('MAPE mean: {} std: {}'.format(erMEan, erStd))

    end = time.time()
    print("Elapsed s: ", end - start)

In [4]:
Run(1e-3)

Step: 0 from: 500000
AVG loss:  1105.4
Step: 10000 from: 500000
AVG loss:  35.3696
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  0.0496223
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  0.014543
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  0.00800985
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  0.0056602
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  0.00434735
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  0.00349455
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  0.003011
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  0.00262776
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  0.00223357
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  0.00201772
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  0.00182588
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.00169829
End of dataset reached.
Step: 140000 from: 500000
AVG loss:  0.0015

In [5]:
#adam
BATCH_SIZE = 100
LEARNING_RATE = 2e-3 #8e-3
startRate = 1e-4
for i in [1e-4, 5e-4, 1e-3, 4e-3, 8e-3, 5e-2, 1e-2,  5e-1, 1e-1, 5e-1]: #4e-3
    print("Learning rate", i)
    Run(i)
    print("------------------")

    


Learning rate 0.0001
Step: 0 from: 500000
AVG loss:  2304.73
Step: 10000 from: 500000
AVG loss:  304.553
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  1.52382
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  0.0837961
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  0.0549668
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  0.0408825
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  0.0319817
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  0.0260096
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  0.0218918
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  0.018857
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  0.017022
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  0.0153979
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  0.0138102
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.0124827
End of dataset reached.
Step: 140000 from: 500000
AVG l

End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.00169079
End of dataset reached.
Step: 140000 from: 500000
AVG loss:  0.00154898
End of dataset reached.
Step: 150000 from: 500000
AVG loss:  0.00145153
Step: 160000 from: 500000
AVG loss:  0.00138673
End of dataset reached.
Step: 170000 from: 500000
AVG loss:  0.00133002
End of dataset reached.
Step: 180000 from: 500000
AVG loss:  0.00122072
End of dataset reached.
Step: 190000 from: 500000
AVG loss:  0.00120515
End of dataset reached.
Step: 200000 from: 500000
AVG loss:  0.00112846
End of dataset reached.
Step: 210000 from: 500000
AVG loss:  0.00108058
End of dataset reached.
Step: 220000 from: 500000
AVG loss:  0.0010153
End of dataset reached.
Step: 230000 from: 500000
AVG loss:  0.000999014
End of dataset reached.
Step: 240000 from: 500000
AVG loss:  0.000933237
End of dataset reached.
Step: 250000 from: 500000
AVG loss:  0.000942748
End of dataset reached.
Step: 260000 from: 500000
AVG loss:  0.000870968
End of datase

Step: 240000 from: 500000
AVG loss:  0.0010452
End of dataset reached.
Step: 250000 from: 500000
AVG loss:  0.000965926
End of dataset reached.
Step: 260000 from: 500000
AVG loss:  0.00095662
End of dataset reached.
Step: 270000 from: 500000
AVG loss:  0.00101314
End of dataset reached.
Step: 280000 from: 500000
AVG loss:  0.00108532
End of dataset reached.
Step: 290000 from: 500000
AVG loss:  0.00102056
End of dataset reached.
Step: 300000 from: 500000
AVG loss:  0.0010154
End of dataset reached.
Step: 310000 from: 500000
AVG loss:  0.00100565
Step: 320000 from: 500000
AVG loss:  0.000979267
End of dataset reached.
Step: 330000 from: 500000
AVG loss:  0.000991879
End of dataset reached.
Step: 340000 from: 500000
AVG loss:  0.000953628
End of dataset reached.
Step: 350000 from: 500000
AVG loss:  0.000983109
End of dataset reached.
Step: 360000 from: 500000
AVG loss:  0.000948069
End of dataset reached.
Step: 370000 from: 500000
AVG loss:  0.00101507
End of dataset reached.
Step: 380000

End of dataset reached.
Step: 370000 from: 500000
AVG loss:  0.0010588
End of dataset reached.
Step: 380000 from: 500000
AVG loss:  0.00105496
End of dataset reached.
Step: 390000 from: 500000
AVG loss:  0.00107975
End of dataset reached.
Step: 400000 from: 500000
AVG loss:  0.0010552
End of dataset reached.
Step: 410000 from: 500000
AVG loss:  0.00107738
End of dataset reached.
Step: 420000 from: 500000
AVG loss:  0.00110999
End of dataset reached.
Step: 430000 from: 500000
AVG loss:  0.00106686
End of dataset reached.
Step: 440000 from: 500000
AVG loss:  0.00104691
End of dataset reached.
Step: 450000 from: 500000
AVG loss:  0.00111061
End of dataset reached.
Step: 460000 from: 500000
AVG loss:  0.00102486
End of dataset reached.
Step: 470000 from: 500000
AVG loss:  0.00106403
Step: 480000 from: 500000
AVG loss:  0.00109062
End of dataset reached.
Step: 490000 from: 500000
AVG loss:  0.00109014
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset rea

Step: 0 from: 500000
AVG loss:  1085.44
Step: 10000 from: 500000
AVG loss:  38.6371
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  83.0282
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  4.48601
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  34.5611
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  4.99272
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  13.2912
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  340.412
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  54.9707
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  0.25147
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  83.7403
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  22.4129
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  26.8321
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  14.268
End of dataset reached.
Step: 140000 from: 500000
AVG loss:  15.6502
End of dataset reached.
Step

In [6]:
#SGD
BATCH_SIZE = 100
LEARNING_RATE = 2e-3 #Learning rate 0.004 - 15,39%
startRate = 1e-4
for i in [1e-4, 5e-4, 1e-3, 4e-3, 8e-3, 5e-2, 1e-2,  5e-1, 1e-1, 5e-1]: #4e-3
    print("Learning rate", i)
    Run(i)
    print("------------------")


Learning rate 0.0001
Step: 0 from: 500000
AVG loss:  1621.73
Step: 10000 from: 500000
AVG loss:  310.209
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  1.56286
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  0.0839435
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  0.0550696
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  0.0405253
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  0.0313627
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  0.0257109
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  0.0221083
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  0.0190451
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  0.0170623
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  0.015391
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  0.0138855
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.0125928
End of dataset reached.
Step: 140000 from: 500000
AVG 

End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.0017233
End of dataset reached.
Step: 140000 from: 500000
AVG loss:  0.00159181
End of dataset reached.
Step: 150000 from: 500000
AVG loss:  0.00150396
Step: 160000 from: 500000
AVG loss:  0.00137247
End of dataset reached.
Step: 170000 from: 500000
AVG loss:  0.00130884
End of dataset reached.
Step: 180000 from: 500000
AVG loss:  0.00125375
End of dataset reached.
Step: 190000 from: 500000
AVG loss:  0.00116439
End of dataset reached.
Step: 200000 from: 500000
AVG loss:  0.00111917
End of dataset reached.
Step: 210000 from: 500000
AVG loss:  0.0010756
End of dataset reached.
Step: 220000 from: 500000
AVG loss:  0.00102093
End of dataset reached.
Step: 230000 from: 500000
AVG loss:  0.000961671
End of dataset reached.
Step: 240000 from: 500000
AVG loss:  0.000959082
End of dataset reached.
Step: 250000 from: 500000
AVG loss:  0.000916321
End of dataset reached.
Step: 260000 from: 500000
AVG loss:  0.000872093
End of dataset

Step: 240000 from: 500000
AVG loss:  0.00101455
End of dataset reached.
Step: 250000 from: 500000
AVG loss:  0.000991237
End of dataset reached.
Step: 260000 from: 500000
AVG loss:  0.00100597
End of dataset reached.
Step: 270000 from: 500000
AVG loss:  0.00101841
End of dataset reached.
Step: 280000 from: 500000
AVG loss:  0.000986012
End of dataset reached.
Step: 290000 from: 500000
AVG loss:  0.000952397
End of dataset reached.
Step: 300000 from: 500000
AVG loss:  0.0009351
End of dataset reached.
Step: 310000 from: 500000
AVG loss:  0.000941045
Step: 320000 from: 500000
AVG loss:  0.000911421
End of dataset reached.
Step: 330000 from: 500000
AVG loss:  0.000950366
End of dataset reached.
Step: 340000 from: 500000
AVG loss:  0.000989697
End of dataset reached.
Step: 350000 from: 500000
AVG loss:  0.00090976
End of dataset reached.
Step: 360000 from: 500000
AVG loss:  0.000995361
End of dataset reached.
Step: 370000 from: 500000
AVG loss:  0.000974822
End of dataset reached.
Step: 38

End of dataset reached.
Step: 370000 from: 500000
AVG loss:  0.0010901
End of dataset reached.
Step: 380000 from: 500000
AVG loss:  0.00119101
End of dataset reached.
Step: 390000 from: 500000
AVG loss:  0.00116796
End of dataset reached.
Step: 400000 from: 500000
AVG loss:  0.00115758
End of dataset reached.
Step: 410000 from: 500000
AVG loss:  0.00115214
End of dataset reached.
Step: 420000 from: 500000
AVG loss:  0.00111945
End of dataset reached.
Step: 430000 from: 500000
AVG loss:  0.00112958
End of dataset reached.
Step: 440000 from: 500000
AVG loss:  0.00109923
End of dataset reached.
Step: 450000 from: 500000
AVG loss:  0.00109645
End of dataset reached.
Step: 460000 from: 500000
AVG loss:  0.00109669
End of dataset reached.
Step: 470000 from: 500000
AVG loss:  0.00107987
Step: 480000 from: 500000
AVG loss:  0.00114126
End of dataset reached.
Step: 490000 from: 500000
AVG loss:  0.00106373
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset re

Step: 0 from: 500000
AVG loss:  980.406
Step: 10000 from: 500000
AVG loss:  8.43292
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  27.7365
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  10.6948
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  34.8134
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  222.053
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  7.02658
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  18.6889
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  33.1524
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  141.326
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  0.685656
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  148.543
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  26.4177
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  5.23184
End of dataset reached.
Step: 140000 from: 500000
AVG loss:  131.327
End of dataset reached.
St

In [8]:
    erMEan, erStd = TrainModel(0.004, trainViewsD, testViewsD, seqLen=SEQUENCE_LENGTH, minPredDelta=119, maxPredDelta=120, 
                               minLen=180, maxLen=None)
    print(erMEan)

Step: 0 from: 500000
AVG loss:  526.291
Step: 10000 from: 500000
AVG loss:  2.91691
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  0.00106158
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  0.00105688
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  0.000964712
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  0.000864809
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  0.000838171
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  0.000855893
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  0.000829709
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  0.000811132
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  0.000831597
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  0.000796631
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  0.000803025
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.00078455
End of dataset reached.
Step: 140000 from: 500000
A

In [9]:
    erMEan, erStd = TrainModel(0.004, trainViewsD, testViewsD, seqLen=7, minPredDelta=1, maxPredDelta=120, 
                               minLen=180, maxLen=None)
    print(erMEan)

Step: 0 from: 500000
AVG loss:  1563.78
Step: 10000 from: 500000
AVG loss:  9.07404
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  0.0246095
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  0.00634517
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  0.00322194
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  0.00219913
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  0.00175186
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  0.00142421
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  0.00118691
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  0.00111706
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  0.00107717
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  0.00100136
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  0.000950455
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.000894892
End of dataset reached.
Step: 140000 from: 500000
AVG loss:

In [20]:
erMEan, erStd = TrainModel_2(trainViewsD, testViewsD, seqLen=SEQUENCE_LENGTH, minPredDelta=119, maxPredDelta=120, 
                               minLen=180, maxLen=None)
print(erMEan)

AVG loss:  490.9
AVG loss:  12.7575
End of dataset reached.
AVG loss:  0.00125744
End of dataset reached.
AVG loss:  0.00123004
End of dataset reached.
AVG loss:  0.00101885
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset reached.
End of dataset reached.


In [21]:
 print(erMEan)

0.277890536579


In [ ]:
erMEan, erStd = TrainModel(0.004, trainViewsD, testViewsD, seqLen=1, minPredDelta=1, maxPredDelta=120, 
                               minLen=180, maxLen=None)
print(erMEan)

Step: 0 from: 500000
AVG loss:  1557.44
Step: 10000 from: 500000
AVG loss:  9.0629
End of dataset reached.
Step: 20000 from: 500000
AVG loss:  0.0237049
End of dataset reached.
Step: 30000 from: 500000
AVG loss:  0.00597783
End of dataset reached.
Step: 40000 from: 500000
AVG loss:  0.00322206
End of dataset reached.
Step: 50000 from: 500000
AVG loss:  0.00222231
End of dataset reached.
Step: 60000 from: 500000
AVG loss:  0.00176627
End of dataset reached.
Step: 70000 from: 500000
AVG loss:  0.00141134
End of dataset reached.
Step: 80000 from: 500000
AVG loss:  0.00120074
End of dataset reached.
Step: 90000 from: 500000
AVG loss:  0.00116707
End of dataset reached.
Step: 100000 from: 500000
AVG loss:  0.00110621
End of dataset reached.
Step: 110000 from: 500000
AVG loss:  0.00102763
End of dataset reached.
Step: 120000 from: 500000
AVG loss:  0.000950973
End of dataset reached.
Step: 130000 from: 500000
AVG loss:  0.000973069
End of dataset reached.
Step: 140000 from: 500000
AVG loss: 

In [12]:
print(erMEan)

0.169140929365
